In [23]:
import tensorflow as tf
from nets.dep_network import depth_estimation
from nets.pose_network import pose_est
from losses import *
#from utils import *
from data_loader import data_loader_with_batch,load_intrinsic

In [26]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
epsilon=0.0000001
@tf.function
def train_step(de,pe,image_left,image_right,intrinsic):
    with tf.GradientTape(persistent=True) as tape:
        dcx7_right,dep_right = de(image_right)
        dcx7_left,dep_left = de(image_left)
        loss_depth = dept_est_loss(dep_left,dep_right,image_left,image_right)
        
        t_left,rot_left = pe(image_left)
        t_right,rot_right = pe(image_right)
        disp = (dep_right[:,:,:,0]+epsilon)
        depth = 1/disp
        pk = tf.slice(image_right,[0,0,0,0],[-1,-1,-1,3])
        pk1 = tf.slice(image_right,[0,0,0,3],[-1,-1,-1,3])
        loss_pose_total = pose_est_loss(rot_right,rot_left,t_left,t_right,pk,pk1,intrinsic,depth)
        #tf.print("loss_pose",loss_pose)

    gradients_de = tape.gradient(loss_depth, de.trainable_variables)
    gradients_pe = tape.gradient(loss_pose_total, pe.trainable_variables)
    optimizer.apply_gradients(zip(gradients_de, de.trainable_variables))
    optimizer.apply_gradients(zip(gradients_pe, pe.trainable_variables))
    return loss_pose_total,loss_depth

In [33]:
EPOCHS = 5
train_ds = []
train_ps = []
#de = depth_estimation()
#pe = pose_est()
batch = 2

##for training images dataset
path_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training"
left_img_dir = path_dir+"/image_2/"
right_img_dir = path_dir+"/image_3/"
img_width = 416
img_height = 128
batch = 2

##for intrinsic the dir is 
calib_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/data_scene_flow_calib/training/calib_cam_to_cam/"
internsic_tensor = load_intrinsic(calib_dir,batch)

train_ds = data_loader_with_batch(left_img_dir,right_img_dir,img_width,img_height,batch,number_of_data=100)

In [ ]:
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    for images_left,images_right in train_ds:
        lp,ld = train_step(de,pe,images_left,images_right,internsic_tensor)

    print("epochs : ",epoch)
    print("loss depth",ld.numpy())
    print("loss pose",lp.numpy())

epochs :  0
loss depth 1.1929013
loss pose 0.56692934
epochs :  1
loss depth 1.1929011
loss pose 0.5935014


In [5]:
_,dep = de(images_right)